In [97]:
import pandas as pd
import numpy as np
import pickle as pkl
import lightgbm as lgb
import xgboost
import re
import time

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import BaseCrossValidator
from sklearn.metrics import  accuracy_score, median_absolute_error, make_scorer

from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.early_stop import no_progress_loss

class CustomTimeSeriesCV(BaseCrossValidator):
    """Creates an iterator that contains the indices from each dataset based on the years given"""
    def __init__(self, years):
        self.years = years

    def split(self, X, y=None, groups=None):
        for train_years, test_years in self.years:
            train_indices = np.where(X['year'].isin(train_years))[0]
            test_indices = np.where(X['year'].isin(test_years))[0]
            yield train_indices, test_indices
        
    def get_n_splits(self, X=None, y=None, groups=None):
        return len(self.years) 
    
def penalize_wrong(y, y_pred, penalty):
    "Penalizes wrong guesses more, determined by the value of k"
    return np.mean(abs((y_pred - y))*(1+penalty*(np.sign(y_pred)
                                               != np.sign(y))))

In [98]:
np.seterr(divide='ignore', invalid='ignore')

#Categorical features that need to be one-hot encoded    
one_hot_fts = ['office_type']

#Rating is the only ordinal feature
ordinal_fts = ['final_rating']
ordinal_fts_ranking = ['Safe R', 'Likely R', 'Leans R', 'Toss-up', 'Leans D', 'Likely D', 'Safe D']

#Cont features that should be pass-throughed (and later scaled)
cont_fts = ['open_seat','incumbent_differential', 'absenteeexcusereq', 'special', 'isMidterm',
       'pollhours', 'avgpollhours', 'minpollhours', 'regdeadlines',
       'voteridlaws', 'novoterid', 'noallmailvote', 'noearlyvote',
       'nofelonreg', 'nofelonsregafterincar', 'nonstrictid', 'nonstrictphoto',
       'nopollplacereg', 'nopr', 'nosamedayreg', 'nostateholiday', 'pr16',
       'pr17', 'pr175', 'pr60', 'pr90', 'strictid', 'strictphoto', 'covi_num',
       'prev_gen_margin', 'prev_dem_gen_tp', 'weighted_genpoll', 'unweighted_genpoll',
       'mean_specials_differential', 'house_chamber_margin',
       'senate_chamber_margin', 'previous_cci', 'current_cci', 'change_cci',
       'previous_gas', 'current_gas', 'change_gas', 'previous_unemployment',
       'current_unemployment', 'change_unemployment', 'receipts_DEM',
       'receipts_REP', 'disbursements_DEM', 'disbursements_REP',
       'unconvinced_pct', 'phone_unweighted', 'online_unweighted', 'num_polls',
       'unweighted_estimate', 'unweighted_ci_lower', 'unweighted_ci_upper',
       'weighted_estimate', 'weighted_ci_lower', 'weighted_ci_upper',
       'white_pct', 'black_pct', 'asian_pct', 'hispanic_pct', 'median_income',
       'impoverished_pct', 'median_age', 'renting_pct', 'inflation',
       'isMidterm', 'receipts_ratio', 'disbursements_ratio', 'total_receipts',
       'total_disbursements', 'genballot_predicted_margin',
       'specials_predicted_margin', 'receipts_genballot_interaction', 'poll_fundamental_agree',
       'disbursements_genballot_interaction', 'gas_democrat_interaction', 'cci_democrat_interaction', 'genballot_predicted_lower',
       'genballot_predicted_upper', 'democrat_in_presidency', 'similar_poll_differential', 'combined_prediction']

def optima_model(model, param_dict, X, y, penalizing_factor = 10):
    """Performs hyperparameter optimization for a a given bootstrapped X 
    ## Parameters:
    model: sklearnable model. We use LGBMRegressor 
    param_dict: dictionary of hyperparameters to optimize
    X: DataFrame with features
    y: Series with target variable"""
    
    X_other, y_other = X.loc[X['year'] <= 2022, :], y.loc[X['year'] <= 2022]
    X_train, X_test, y_train, y_test = (X.loc[X['year'] < 2022, :], X.loc[X['year'] == 2022, :], 
                                        y.loc[X['year'] < 2022], y.loc[X['year'] == 2022])
    
    # Create fold structure so we can make a custom cross-validation for time-series
    folds = [
        (range(2002, 2006, 2), [2006, 2008]),
        (range(2002, 2010, 2), [2010, 2012]),
        (range(2002, 2014, 2), [2014, 2016]),
        (range(2002, 2018, 2), [2020])
    ]

    cv = CustomTimeSeriesCV(folds)
        
    #Preprocessing data: no need to scale data, because we use tree-based models which are monotonic-scale-invariant
    #Because we don't need to scale data, we don't have to include the column transformer in the final saved model
    preprocessor = ColumnTransformer([
        ('cat', OneHotEncoder(), one_hot_fts),
        ('ord', OrdinalEncoder(categories = [ordinal_fts_ranking], handle_unknown='use_encoded_value', 
                               unknown_value=np.nan), ordinal_fts),
        ('num', 'passthrough', cont_fts)])
    
    def objective(params):
        print("Params: ", params)
        "Function that takes in hyperparameters and returns loss, that Hyperopt will minimize."        
        testing_loss = [] 
        accuracies = []
        for train_idx, test_idx in cv.split(X_train):
            reg = model(**params)
            pipe = Pipeline(steps = [
                ('preprocessing', preprocessor), 
                ('model', reg)])
            
            
            #Necessary steps to utilize early-stopping on LGBM
            pipe.named_steps['preprocessing'].fit(X_train.iloc[train_idx])
            transformed_val = pipe.named_steps['preprocessing'].transform(X_train.iloc[test_idx])
            penalize_scorer = make_scorer(penalize_wrong, greaterisbetter=False, penalizing_factor = penalizing_factor)

            
            """Goes through each fold and calculates loss.
            Note: We use median absolute error because it is more robust to outliers than mean absolute error.
            We also expect earlier folds to have higher error, since they have less data to train on."""
            early_stopping = lgb.early_stopping(10, verbose = False)
            
            pipe.fit(X_train.iloc[train_idx], y_train.iloc[train_idx],
                     model__eval_set = [(transformed_val, y_train.iloc[test_idx])], model__eval_metric = 'mae', 
                     model__callbacks = [early_stopping])            
            
            
            predictions = pipe.predict(X_train.iloc[test_idx])
            testing_loss.append(penalize_wrong(y_train.iloc[test_idx], predictions, penalizing_factor))
            accuracies.append(accuracy_score(np.sign(y_train.iloc[test_idx]), np.sign(predictions)))
            
        mean_test_loss = np.mean(testing_loss)
        print(f"Validation loss: {testing_loss}, mean: {mean_test_loss}")
        print(f"Validation accuracy: {accuracies}, mean: {np.mean(accuracies)}")
        return {'loss': mean_test_loss, 'status': STATUS_OK}

    start_time = time.time()
    max_time = 20 #about two minutes per run-through
    def stop(trial, elapsed_time=0):
        return elapsed_time > max_time, [time.time() - start_time] 
    
    "Hyperopt uses the TPE algorithm to optimize hyperparameters. We use the no_progress_loss function to stop early if we don't see progress."
    best_params = fmin(fn=objective,
                    space=param_dict,
                    algo=tpe.suggest,
                    trials=Trials(),
                    early_stop_fn = stop)
                    
                    
    print("Best parameters:", best_params)
    best_model = model(**best_params)
    pipe = Pipeline(steps = [
        ('preprocessing', preprocessor), 
        ('model', best_model)])
    
    #Returns a fitted ML algortithm with those hyperparameters
    pipe.fit(X_other, y_other) 
    #Returns the final model   
    return pipe.named_steps['model']

In [99]:
#Bootstraps X and y, and then runs the optimization function
def bootstrap(group, n=None):
    if n is None:
        n = len(group)
    return group.sample(n, replace=True)

data = pd.read_csv("../cleaned_data/Engineered Dataset.csv")
data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

preprocessor = ColumnTransformer([
        ('cat', OneHotEncoder(), one_hot_fts),
        ('ord', OrdinalEncoder(categories = [ordinal_fts_ranking], handle_unknown='use_encoded_value', 
                               unknown_value=np.nan), ordinal_fts),
        ('num', 'passthrough', cont_fts)])

names_for_monotonicity = preprocessor.fit(data.drop(columns=['margin'])).get_feature_names_out()
before_processing_monotonic_columns = ['incumbent_differential', 'pvi', 'receipts_ratio', 'disbursements_ratio', 
                                       'genballot_predicted_margin', 'specials_predicted_margin', 'unweighted_estimate', 'unweighted_ci_lower',
                                       'unweighted_ci_upper','weighted_estimate', 'weighted_ci_lower', 'weighted_ci_upper',
                                       'phone_unweighted', 'online_unweighted', 'receipts_genballot_interaction',
                                       'disbursements_genballot_interaction', 'poll_fundamental_average']

monotonic_columns = ['num__' + name for name in before_processing_monotonic_columns]
monotone_constraints = [1 if name in monotonic_columns else 0 for name in names_for_monotonicity]

# Define the search space for Hyperopt
param_dist_lgbm = {
    'boosting_type': 'gbdt',  # Removed 'goss' to simplify
    'num_leaves': hp.randint('num_leaves', 20, 70),  # Reduced the upper limit, 
    'learning_rate': hp.loguniform('learning_rate', -6, -1),  # Equivalent to about 0.0001 to 0.01
    'subsample_for_bin': hp.randint('subsample_for_bin', 20000, 200000),  # Narrowed the range
    'min_data_in_bin': hp.randint('min_data_in_bin', 1, 10), 
    'min_data_in_leaf': hp.randint('min_data_in_leaf', 1, 10),  # Reduced the upper limit
    'min_child_samples': hp.randint('min_child_samples', 20, 150),  # Increased the range for more regularization
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 2.5),  # Increased upper limit for L1 regularization
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 2.5),  # Increased upper limit for L2 regularization
    'colsample_bytree': hp.uniform('colsample_bytree', 0.4, 0.8),  # Reduced the upper limit
    'subsample': hp.uniform('subsample', 0.5, 0.8),  # Reduced the upper limit for more randomness
    'max_depth': hp.randint('max_depth', 2, 15),  # Added max_depth for additional control
    "verbose": -1,  # Keep verbose to -1 to reduce log clutter, 
    'monotone_constraints': monotone_constraints
}


#optima_model(lgb.LGBMRegressor, param_dist_lgbm, data.drop(columns=['margin']), data['margin'])

num_trials = 2
for idx in range(num_trials):
    bootstrapped_data = data.groupby(['year', 'office_type']).apply(bootstrap).reset_index(drop=True)
    
    bootstrapped_X = bootstrapped_data.drop(columns=['margin'])
    bootstrapped_y = bootstrapped_data['margin']
    
    trained_lgbm = optima_model(lgb.LGBMRegressor, param_dist_lgbm, bootstrapped_X, bootstrapped_y)
    file_path = f"../models/Model_{idx}.pkl"

    # Open a file to write in binary mode????        
    with open(file_path, 'wb') as file:
        pkl.dump(trained_lgbm, file)


Params:                                                                
{'boosting_type': 'gbdt', 'colsample_bytree': 0.5946499761983302, 'learning_rate': 0.12677193009772983, 'max_depth': 6, 'min_child_samples': 149, 'min_data_in_bin': 9, 'min_data_in_leaf': 8, 'monotone_constraints': (0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0), 'num_leaves': 36, 'reg_alpha': 0.5728368538425305, 'reg_lambda': 2.0121595466177173, 'subsample': 0.7129112953810639, 'subsample_for_bin': 23745, 'verbose': -1}
Validation loss: [16.75447591389995, 12.759005065600467, 12.042767254375418, 12.906633239397518], mean: 13.615720368318339
Validation accuracy: [0.9420131291028446, 0.950836820083682, 0.9537366548042705, 0.936734693877551], mean: 0.945830324467087
Params:                         